In [1]:
import torch
import torch.autograd as autograd
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import string
import time
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

%load_ext autoreload
%autoreload 2

from rnn import RNN, PrepareData

In [2]:
#input_dim = 7
hidden_dim = 30
num_layers = 2
seq_size = 25
all_letters = string.printable
n_letters = len(all_letters)

In [3]:
#dataset = PrepareData('input.txt', seq_size, val_ratio=0.2)
dataset = PrepareData('sample-music2.txt', seq_size)
train_data, val_data = dataset.dataset()

In [51]:
def inputTensor(line):
    if len(line) == 1:
        tensor = torch.zeros(1, n_letters)
        tensor[0, all_letters.find(line[0])] = 1
    else:
        tensor = torch.zeros(len(line)-1, n_letters)
        for idx, letter in enumerate(line[:-1]):
            tensor[idx, all_letters.find(letter)] = 1
    return Variable(tensor.cuda())
        
def targetTensor(line):
    letter_indexes = [all_letters.find(letter) for letter in line[1:]]
    return Variable(torch.LongTensor(letter_indexes).cuda())

def save_model(model, path):
    torch.save(model.state_dict(), path)
    
def load_model(model, path):
    model.load_state_dict(torch.load(path))

In [5]:
model = RNN(n_letters, hidden_dim, num_layers=num_layers, output_size=n_letters)
model = model.cuda()
criterion = nn.CrossEntropyLoss()

In [7]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
#scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)

In [57]:
n_epochs = 20
all_losses = []

print_every = 500
init_hidden_every = 20

start_time = time.time()

for epoch in range(n_epochs):
    train_cnt = 0
    total_loss = 0.
    
    model.hidden = model.init_hidden()
    
    #dataset.shuffle_data()
    #train_data, val_data = dataset.dataset()
    
    for sentence in train_data:
        model.zero_grad()
        
        loss = 0.
        inputs = inputTensor(sentence)
        targets = targetTensor(sentence)
        
        preds = model(inputs)
        loss = criterion(preds, targets)
            
        loss.backward(retain_graph=True)
        optimizer.step()
        total_loss += loss
        train_cnt += 1
        
        if train_cnt % init_hidden_every == 0:
            model.hidden = model.init_hidden()
        
        if train_cnt % print_every == 0:
            total_loss /= print_every
            all_losses.append(total_loss)
            uptime = time.time() - start_time
            print('%dm:%s -- [%d, %d], loss: %f' % (uptime/60, uptime%60, epoch, train_cnt, total_loss))
            total_loss = 0
#scheduler.step()

0m:9.978902339935303 -- [0, 500], loss: 1.993074
0m:20.233942985534668 -- [0, 1000], loss: 1.949602
0m:36.09587645530701 -- [1, 500], loss: 1.986950
0m:44.97156286239624 -- [1, 1000], loss: 1.940296
1m:0.21548199653625488 -- [2, 500], loss: 1.983391
1m:9.102586269378662 -- [2, 1000], loss: 1.935963
1m:24.32178235054016 -- [3, 500], loss: 1.974745
1m:33.17969059944153 -- [3, 1000], loss: 1.932875
1m:48.407769203186035 -- [4, 500], loss: 1.970787
1m:57.28315997123718 -- [4, 1000], loss: 1.922637
2m:12.574362754821777 -- [5, 500], loss: 1.968722
2m:21.435510635375977 -- [5, 1000], loss: 1.918855
2m:37.609251976013184 -- [6, 500], loss: 1.961254
2m:48.278090476989746 -- [6, 1000], loss: 1.915207
3m:7.0418901443481445 -- [7, 500], loss: 1.956862
3m:17.701841831207275 -- [7, 1000], loss: 1.912813
3m:36.33969974517822 -- [8, 500], loss: 1.952200
3m:47.2464542388916 -- [8, 1000], loss: 1.904044
4m:5.684267997741699 -- [9, 500], loss: 1.941889
4m:16.473621368408203 -- [9, 1000], loss: 1.898563


In [43]:
def test(start_with, max_length):
    model.hidden = model.init_hidden()
    seq = str(start_with)
    idx = []
    inputs = inputTensor(seq[-1])
    
    for i in range(max_length):
        output = model(inputs)
        _, pred = torch.max(output.data, 1)
        
        seq += all_letters[pred[0]]
        idx.append(pred[0])
        inputs = inputTensor(letter)
    return seq, idx

In [58]:
test_seq = train_data[5]

seq, idx = test(test_seq[7],25)
print('Generated sequence: %s' % (seq))
print('Sequence index: ', idx)
print('Test sequence: %s' % test_seq)

Generated sequence: M BBBBB|Bccccccccccccccccc
Sequence index:  [94, 37, 37, 37, 37, 37, 91, 37, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12]
Test sequence: ree.fr
M: 4/4
L: 1/8
Q:1/


In [54]:
save_model(model, 'sample-model.pth')

In [ ]:
plt.figure()
plt.plot(all_losses)